In [1]:
import os
from os import environ
from typing import cast
from dotenv import load_dotenv, find_dotenv
import getpass

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.language_models import LanguageModelInput
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.embeddings import GigaChatEmbeddings
from langchain_core.messages import BaseMessage

## Примеры использования `tagme_ai_traces`


## Оглавление

[Примеры использования tagme\_ai\_traces](#scrollTo=d6eddca6)

> [Оглавление](#scrollTo=293c899b)

> [init](#scrollTo=a945d530) — установка, переменные окружения, проверка токенов.

> [Клиент TagMeTrace](#scrollTo=37d22962) — низкоуровневый HTTP-клиент для отправки/получения данных.
>
> > [1.1. Отправка диалогов](#scrollTo=e55f7bba) — как упаковать `DialogData` и отправить.
> >
> > [1.2. Отправка описания функции (tool / инструмента)](#scrollTo=8820b48a) — регистрация `FunctionDef` для function-calling.
> >
> > [1.3. Получение статистики и результатов разметки](#scrollTo=bcb8ea20) — чтение метрик/результатов для анализа.

> [Использование TagMeAgentTracer - langchain callback handler](#scrollTo=84e58084) — автоматический сбор диалогов и функций из коллбеков.
>
> > [Пример 1: Интерактивный агент (ReAct) с последовательной историей диалога](#scrollTo=7c989c20) — сохраняем весь ход агентного шага.
> >
> > [Пример 2: Вызов функции-инструмента внутри диалога](#scrollTo=31364e9e) — трассируем function call → tool result → финальный ответ.
> >
> > [Пример 3: Интеграция трейсера в RAG-цепочку (Retrieval-Augmented Generation)](#scrollTo=4bcfc7ad) — подключение к `retriever|prompt|LLM`.

> [Использование декораторов на примере цепочки RAG для генерации ответа модели с использованием RAG](#scrollTo=06cdc7be) — как обернуть вызовы и слать данные автоматически.
>
> > [3.1 Асинхронный декоратор](#scrollTo=669e177d) — для `.ainvoke()`.
> >
> > [3.2. Синхронный декоратор](#scrollTo=6f361025) — для `.invoke()`.
> >
> > [Функция для преобразования диалога](#scrollTo=6486cfc0) — если оборачиваем что-то нетипичное.


### init


Установка `tagme_ai_traces`


In [ ]:
# %pip install ..

Для работы нужны базовый url и токен.

Можем передавать в клиенты, декораторы и трейсеры как аргументы, а можем указать переменные окружения TAGME_TOKEN и TAGME_BASE_URL

<span style="color: gray">*токен в примерах тестовый, и не будет работать. для получения собственного токена обратитесь к dmpresnukhin@sberbank.ru*</span>

In [ ]:
load_dotenv(find_dotenv())

token = os.environ.get("TAGME_TOKEN", "d6594992-4348-44f7-a5b6-6092d11730e6")
base_url = os.environ.get("TAGME_BASE_URL", "https://tagme.sberdevices.ru/dev/chatwm/plugin_statistics/trace")

Можем в юпитере вот так прописать env


In [ ]:
%set_env TAGME_TOKEN=d6594992-4348-44f7-a5b6-6092d11730e6
%set_env TAGME_BASE_URL=https://tagme.sberdevices.ru/dev/chatwm/plugin_statistics/trace

env: TAGME_TOKEN=d6594992-4348-44f7-a5b6-6092d11730e6
env: TAGME_BASE_URL=http://localhost:8000/


В качестве подопытной LLM конечно же используем GigaChat. В приведенных примерах для доступа я использую токен


In [ ]:
if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Введите ключ авторизации GigaChat API: ")

Для наглядности выведем логи на уровень INFO (чтобы видеть сообщения logger.info, которые появляются при сохранении диалога, и при необходимости можно установить DEBUG, но там будет много данных от запросов к API)


In [ ]:
import logging

# logging.basicConfig(level=logging.DEBUG, format="[%(levelname)s](%(name)s): %(message)s")
logging.basicConfig(level=logging.INFO, format="[%(levelname)s](%(name)s): %(message)s")

### 1. Клиент TagMeTrace <a name="client"></a>

Базовый клиент для отправки данных - TagmeIntegrationClientAsync и его синхронная версия TagmeIntegrationClientSync


In [ ]:
from tagme_ai_traces import TagmeIntegrationClientAsync  # , TagmeIntegrationClientSync


tagme_trace_client = TagmeIntegrationClientAsync(
    token=token,  # передаём токен или используем env TAGME_TOKEN
    base_url=base_url,  # передаём base_url или используем env TAGME_BASE_URL
    ignore_missing_functions=True,  # не вызываеть ошибки, если в диалоге содержится функция, описание которой заранее не было отправлено на сервер
)

#### 1.1. Отправка диалогов

Клиент принимает диалоги в виде датаклассов `DialogData`


In [ ]:
from tagme_ai_traces.entities import DialogData


dialog_delete_alarm = DialogData.from_dict(
    {
        "input": [
            {"role": "system", "content": "You are a smart home assistant."},
            {"role": "user", "content": "Удалите будильник на 7 утра."},
            {
                "role": "assistant",
                "content": None,
                "function_call": {"name": "delete_alarm_unknown", "arguments": {"time": "07:00"}},
            },
            {
                "role": "function",
                "content": None,
                "function_result": {"name": "delete_alarm_unknown", "result": {"status": "success"}},
            },
            {"role": "assistant", "content": "Будильник на 7:00 удалён."},
        ],
        "metadata": {"test": "delete_alarm_unknown"},
    }
)

In [ ]:
dialog_delete_alarm.input

[ChatMessage(role='system', content='You are a smart home assistant.', function_call=None, function_result=None),
 ChatMessage(role='user', content='Удалите будильник на 7 утра.', function_call=None, function_result=None),
 ChatMessage(role='assistant', content=None, function_call=FunctionCall(name='delete_alarm_unknown', arguments={'time': '07:00'}), function_result=None),
 ChatMessage(role='function', content=None, function_call=None, function_result=FunctionResult(name='delete_alarm_unknown', result={'status': 'success'})),
 ChatMessage(role='assistant', content='Будильник на 7:00 удалён.', function_call=None, function_result=None)]

In [ ]:
await tagme_trace_client.send_dialog(dialog_delete_alarm)

{'id': '2de5dc36-0b47-464c-81b7-c7f0c718db22',
 'missing': ['delete_alarm_unknown'],
 'linked_functions': [],
 'warning': True}

Получили в выводе информацию, что мы передали диалог с вызовом функции, которой нет в диалоге. К сожалению, разметчики уже не увидят информацию про эту функцию. Все определения нужно отправлять заранее


#### 1.2. Отправка описания функции (tool / инструмента)


In [ ]:
function_delete_alarm = {
    "name": "delete_alarm_unknown",
    "description": "Функция удаления будильника по id. Данная функция запускается только при наличии необходимых идентификаторов id будильников в контексте. Если пользователь явно не передал id будильника, то получи метаинформацию об установленных будильниках, вызвав сначала соответствую функцию и только затем используй функцию удаления по id. Если пользователь просит удалить все будильники и в контексте диалога есть необходимые id или пользователь явно передает id будильника, который надо удалить, то вызови эту функцию, переспрашивать пользователя не нужно. В остальных случаях, при наличии необходимых id в контексте диалога и готовности удалить будильник, сначала переспроси пользователя подтверждает ли он удаление будильника и вызывай функцию только при наличии подтверждения от пользователя.",
    "parameters": {
        "properties": {
            "ids": {
                "description": "Список id будильников, которые нужно удалить",
                "items": {"description": "Идентификатор id будильника, который нужно удалить", "type": "string"},
                "type": "array",
            }
        },
        "required": ["ids"],
        "type": "object",
    },
    "return_parameters": {
        "description": "Ответ на delete_alarm",
        "properties": {
            "error": {"description": "Текст ошибки в случае, если status == fail", "type": "string"},
            "ids": {
                "description": "Список id будильников, которые удалились",
                "items": {"description": "Идентификатор id будильника, который удалился", "type": "string"},
                "type": "array",
            },
            "status": {
                "description": "Статус - удалось ли удалить будильник.",
                "type": "string",
                "enum": ["success", "fail"],
            },
        },
        "required": ["status"],
        "type": "object",
    },
    "few_shot_examples": [],
}

In [ ]:
from tagme_ai_traces.entities import FunctionDef


await tagme_trace_client.send_functions(functions=[FunctionDef.from_dict(function_delete_alarm)])

{'results': [{'name': 'delete_alarm_unknown',
   'id': 5,
   'version': 1,
   'status': 'created'}]}

Отлично, функция отправлена, теперь в последующих диалогах разметчики будут видеть её описание


Отправим ещё один диалог с этой функцией


In [ ]:
await tagme_trace_client.send_dialog(dialog_delete_alarm)

{'id': '25e0d234-7cd5-470d-9162-fead63ec572e',
 'missing': [],
 'linked_functions': [{'id': 5, 'name': 'delete_alarm_unknown', 'version': 1}],
 'warning': False}

Видим, что к новому диалогу привязалась функция, и теперь разметчики увидят её описание


In [ ]:
from tagme_ai_traces import ChatMessage


function_get_weather = FunctionDef.from_dict(
    {
        "name": "get_weather",
        "description": "Получение погоды в городе",
        "parameters": {
            "properties": {
                "city": {
                    "description": "Название города",
                    "type": "string",
                }
            },
            "required": ["city"],
            "type": "object",
        },
    }
)

await tagme_trace_client.send_functions([function_get_weather])

{'results': [{'name': 'get_weather',
   'id': 6,
   'version': 1,
   'status': 'created'}]}

In [ ]:
new_dialog = DialogData(
    input=[
        ChatMessage.from_dict({"role": "user", "content": "Какая погода?"}),
        ChatMessage.from_dict(
            {
                "role": "assistant",
                "content": None,
                "function_call": {"name": "get_weather", "arguments": {"city": "Moscow"}},
            }
        ),
        ChatMessage.from_dict(
            {
                "role": "function",
                "content": None,
                "function_result": {"name": "get_weather", "result": {"temperature": 20, "condition": "sunny"}},
            }
        ),
        ChatMessage.from_dict({"role": "assistant", "content": "В Москве сейчас 20 градусов и солнечно."}),
    ]
)
await tagme_trace_client.send_dialog(new_dialog)

{'id': '78061dd0-f3ef-45a3-bbfd-b74670854218',
 'missing': [],
 'linked_functions': [{'id': 6, 'name': 'get_weather', 'version': 1}],
 'warning': False}

Можем получить список всех функций, которые мы загружали


In [ ]:
await tagme_trace_client.get_functions()

[FunctionResponse(id=5, name='delete_alarm_unknown', version=1, definition={'description': 'Функция удаления будильника по id. Данная функция запускается только при наличии необходимых идентификаторов id будильников в контексте. Если пользователь явно не передал id будильника, то получи метаинформацию об установленных будильниках, вызвав сначала соответствую функцию и только затем используй функцию удаления по id. Если пользователь просит удалить все будильники и в контексте диалога есть необходимые id или пользователь явно передает id будильника, который надо удалить, то вызови эту функцию, переспрашивать пользователя не нужно. В остальных случаях, при наличии необходимых id в контексте диалога и готовности удалить будильник, сначала переспроси пользователя подтверждает ли он удаление будильника и вызывай функцию только при наличии подтверждения от пользователя.', 'parameters': {'properties': {'ids': {'description': 'Список id будильников, которые нужно удалить', 'items': {'descriptio

#### 1.3. Получение статистики и результатов разметки


Клиент поддерживает ряд методов для получения результатов разметки и аггрегированных результатов.

Сейчас здесь данные из тестового проекта, реальные оцениваемые в диалоге метрики будут отличаться. Преобразование в датаклассы и фильтрация пока в разработке


Получение метрик разметки по дням


In [ ]:
await tagme_trace_client.get_markup_quality()

[{'date': '2025-09-23T00:00:00Z',
  'metrics': {'beauty': 1.0,
   'literacy': 1.5,
   'criteria_1': 1.0,
   'criteria_2': 1.0,
   'criteria_3': 1.0,
   'impression': 5.0}}]

Получние статистики размеченных диалогов


In [ ]:
await tagme_trace_client.get_markup_statistics()

{'task_id': '997584b8-25a5-4948-9235-f41022c979d5',
 'total_dialogs': 4,
 'marked_dialogs': 2,
 'total_markups': 2,
 'accepted_markups': 2,
 'overlap': 1}

Получение сырых результатов разметки


In [ ]:
await tagme_trace_client.get_results()

[{'assignment_id': 'dd0f7218-ae2c-4c65-985e-90231857198c',
  'dialog_id': '2de5dc36-0b47-464c-81b7-c7f0c718db22',
  'payload': {'beauty': '1',
   'output': [{'role': 'system',
     'content': 'You are a smart home assistant.',
     'function_call': None,
     'function_result': None},
    {'role': 'user',
     'content': 'Удалите будильник на 7 утра.',
     'function_call': None,
     'function_result': None},
    {'role': 'assistant',
     'content': None,
     'function_call': {'name': 'delete_alarm_unknown',
      'arguments': {'time': '07:00'}},
     'function_result': None},
    {'role': 'function',
     'content': None,
     'function_call': None,
     'function_result': {'name': 'delete_alarm_unknown',
      'result': {'status': 'success'}}},
    {'role': 'assistant',
     'content': 'Будильник на 7:00 удалён.',
     'function_call': None,
     'function_result': None}],
   'comment': 'Всё верно',
   'literacy': '2',
   'criteria_1': '1',
   'criteria_2': '1',
   'criteria_3': '

### 2. Использование `TagMeAgentTracer` - langchain callback handler

Удобнее всего для отправки использовать `TagMeAgentTracer` - асинхронный обработчик событий. Он работает автоматически в разных сценариях: как при использовании агентов, так и при обычных вызовах `ainvoke`


In [ ]:
from tagme_ai_traces import TagMeAgentTracer

#### Пример 1: Интерактивный агент (ReAct) с последовательной историей диалога <a name="callbacks"></a>

Здесь мы будем использовать готовый ReAct-агент на базе GigaChat, который может планировать действия (вызовы инструментов) и отвечать пользователю.

Для этого создадим агент с помощью утилиты `create_react_agent` из библиотеки `langgraph` и подключим к нему наш трейсер. Агенту передаются инструменты (например, функция `get_cats_info`) и механизм сохранения памяти (MemorySaver), чтобы он помнил предыдущие сообщения.


In [ ]:
# Инициализация трейсера (verbose_send=True чтобы видеть событие сохранения на уровне INFO)
agent_tracer = TagMeAgentTracer(
    verbose_send=True,
    client=tagme_trace_client,  # можем или передать client напрямую или args/kwargs для него
)

In [ ]:
from langchain.tools import tool


@tool
async def get_cats_info() -> str:
    """Возвращает справочную информацию о кошках."""
    print('\x1b[32m"Вызвана функция get_cats_info"\x1b[0m')
    return "Кошки известны своей независимостью и грациозностью."

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver


system_prompt = "Отвечай так, будто ты очень любишь кошек."

# создадим model без коллбеков
model_agent = GigaChat(model="GigaChat-2-Max", verify_ssl_certs=False, scope="GIGACHAT_API_CORP", callbacks=[])
agent = create_react_agent(model_agent, tools=[get_cats_info], checkpointer=MemorySaver(), prompt=system_prompt)


cfg: dict = {
    "run_name": "react_agent_demo",
    "tags": ["langgraph", "demo"],
    "metadata": {"env": "local"},  # произвольные метаданные
    "callbacks": [agent_tracer],  # подключаем трейсер в callbacks агента
}

**Важно!** Если в `config` останется `metadata.thread_id` при повторном запуске, то трейсер будет думать, что это всё тот же старый диалог, даже если в `configurable` новый uid. Трейсер при этом перезапишет все старые сообщения новыми и история и не сохранится, и не отправится.

Если это нежелательное поведение, то следует убирать `thread_id` из `metadata` между разными диалогами или перезаписывать новым значением (как на примере)


In [ ]:
import asyncio
import uuid

messages = []


async def chat(thread_id: str):
    print(f"{thread_id = }")
    # обязательно объединяем базовый config с уникальным thread_id для этой сессии
    config = cfg | {"configurable": {"thread_id": thread_id}}

    # Дублируем thread_id в metadata, чтобы убедиться, что в трейсер попадёт корректный идентификатор
    if methadata := config.get("metadata"):
        methadata["thread_id"] = thread_id

    while True:
        rq = input("\nHuman: ")
        print("User: ", rq)
        if rq == "":
            break
        user_msg = ("user", rq)
        messages.append(user_msg)
        resp = await agent.ainvoke({"messages": [user_msg]}, config=config)
        messages.append(resp)
        print("Assistant: ", resp["messages"][-1].content)
        await asyncio.sleep(1)


await chat(str(uuid.uuid4()))

thread_id = '792f964e-cd3a-455c-b75e-c0909d357699'


[INFO](httpx): HTTP Request: POST https://ngw.devices.sberbank.ru:9443/api/v2/oauth "HTTP/1.1 200 OK"


User:  Привет


[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


Assistant:  Мррр, привет! А у тебя есть пушистый друг? Кошки — это самые замечательные создания на свете! Их мягкая шерстка и милые ушки просто очаровывают с первого взгляда. Расскажи мне больше про свою котейку или кота мечты!
User:  Что ты знаешь о кошках? 


[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


"Вызвана функция get_cats_info"


[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


Assistant:  О-о-о, кошки - это такие удивительные существа! Они настолько изящны в своих движениях, что наблюдать за ними можно бесконечно долго. Эти маленькие охотники обладают невероятной ловкостью и потрясающей способностью адаптироваться к любым условиям жизни. Но самое главное их достоинство – это независимость. Нет ничего приятнее, чем чувствовать себя любимым именно тогда, когда кошка сама решит проявить нежность. Просто прелесть, правда?!
User:  


Видим, как трейсер сохранил все сообщения, в том числе вызовы функций


In [ ]:
agent_tracer.dialog

[ChatMessage(role='system', content='Отвечай так, будто ты очень любишь кошек.', function_call=None, function_result=None),
 ChatMessage(role='user', content='Привет', function_call=None, function_result=None),
 ChatMessage(role='assistant', content='Мррр, привет! А у тебя есть пушистый друг? Кошки — это самые замечательные создания на свете! Их мягкая шерстка и милые ушки просто очаровывают с первого взгляда. Расскажи мне больше про свою котейку или кота мечты!', function_call=None, function_result=None),
 ChatMessage(role='user', content='Что ты знаешь о кошках? ', function_call=None, function_result=None),
 ChatMessage(role='assistant', content=None, function_call=FunctionCall(name='get_cats_info', arguments={}), function_result=None),
 ChatMessage(role='function', content='Кошки известны своей независимостью и грациозностью.', function_call=None, function_result=None),
 ChatMessage(role='assistant', content='О-о-о, кошки - это такие удивительные существа! Они настолько изящны в сво

Можем сделать ручной `await agent_tracer.flush()`, или просто запустить новый диалог с новым `thread_id`


In [ ]:
await chat(str(uuid.uuid4()))

thread_id = '8515c658-8e6e-4fee-9d9f-1670de986418'


[INFO](tagme_gigachain.tracer): Sending 1 function definitions (thread_id=792f964e-cd3a-455c-b75e-c0909d357699)
[INFO](tagme_gigachain.tracer): Sending 7 chat messages (thread_id=792f964e-cd3a-455c-b75e-c0909d357699)


User:  Привет


[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


Assistant:  Мяу-привет! Ты готов к обнимашкам с пушистиками? Давай поговорим про котиков и их милые проделки!
User:  


Видим, как при начале нового диалога отправилось 7 предыдущих сообщений + описани функции


Самый последний диалог отправим вручную через `.flush()`


In [ ]:
await agent_tracer.flush()

[INFO](tagme_gigachain.tracer): Sending 1 function definitions (thread_id=8515c658-8e6e-4fee-9d9f-1670de986418)
[INFO](tagme_gigachain.tracer): Sending 3 chat messages (thread_id=8515c658-8e6e-4fee-9d9f-1670de986418)


#### Пример 2: Вызов функции-инструмента внутри диалога

Рассмотрим простой диалог, в котором пользователь спрашивает: "Расскажи о кошках коротко". Мы подготовим функцию-инструмент (tool), которая предоставляет дополнительную информацию о кошках, и позволим модели вызвать эту функцию для получения подсказки перед формированием ответа пользователю.


In [ ]:
tracer = TagMeAgentTracer(verbose_send=True, client=tagme_trace_client)

В GigaChat(...) мы передаём список callbacks, включая наш tracer, чтобы он автоматически получал уведомления о ходе диалога.


In [ ]:
model = GigaChat(model="GigaChat-2-Max", verify_ssl_certs=False, scope="GIGACHAT_API_CORP", callbacks=[tracer])

Тут функция вызывается нами вручную, я не с помощью агента


In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

model_with_tool = model.bind_tools([get_cats_info])

In [ ]:
# type: ignore


async def make_a_dialog():
    messages = [HumanMessage(content="расскажи о кошках коротко")]

    assistant_reply = await model_with_tool.ainvoke(messages)

    # Если модель решила вызвать функцию, обработаем вызов
    if assistant_reply.tool_calls:
        # Добавляем сгенерированный моделью запрос на функцию в историю диалога
        messages.append(assistant_reply)
        for tool_call in assistant_reply.tool_calls:
            # Выполняем реальную функцию get_cats_info и получаем результат
            result = await get_cats_info.ainvoke(tool_call.get("args", {}) or {})
            # Добавляем ответ инструмента в виде ToolMessage
            messages.append(ToolMessage(content=str(result), tool_call_id=tool_call["id"], name=tool_call["name"]))
        assistant_reply = await model_with_tool.ainvoke(messages)

    print("Ответ ассистента:", assistant_reply.content)

In [ ]:
await make_a_dialog()

[INFO](httpx): HTTP Request: POST https://ngw.devices.sberbank.ru:9443/api/v2/oauth "HTTP/1.1 200 OK"
[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


"Вызвана функция get_cats_info"


[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


Ответ ассистента: Кошки — это домашние животные, известные своей самостоятельностью и элегантностью движений. Они ценятся людьми за свою привязанность, охотничьи инстинкты и способность дарить уют своим хозяевам.


При обычных вызовах через invoke тяжело понять, это новая цепочка диалогов или нет, поэтому требуется вручную вызвать await tracer.flush() для отправки данных


In [ ]:
tracer.dialog

[ChatMessage(role='user', content='расскажи о кошках коротко', function_call=None, function_result=None),
 ChatMessage(role='assistant', content=None, function_call=FunctionCall(name='get_cats_info', arguments={}), function_result=None),
 ChatMessage(role='function', content='Кошки известны своей независимостью и грациозностью.', function_call=None, function_result=None),
 ChatMessage(role='assistant', content='Кошки — это домашние животные, известные своей самостоятельностью и элегантностью движений. Они ценятся людьми за свою привязанность, охотничьи инстинкты и способность дарить уют своим хозяевам.', function_call=None, function_result=None)]

In [ ]:
await tracer.flush()

[INFO](tagme_gigachain.tracer): Sending 1 function definitions (thread_id=fc2a8868-ffbf-4315-abb9-631d443ad2e0)
[INFO](tagme_gigachain.tracer): Sending 4 chat messages (thread_id=fc2a8868-ffbf-4315-abb9-631d443ad2e0)


In [ ]:
tracer.dialog

[]

Видим, что отправились два 4 и одна функция


#### Пример 3: Интеграция трейсера в RAG-цепочку (Retrieval-Augmented Generation)


##### Добавим трейсер в callbacks


In [ ]:
model = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_CORP", callbacks=[tracer])

##### Сформируем цепочку


In [ ]:
# Создание документов
documents = [
    Document(page_content="Собаки — отличные компаньоны, известные своей преданностью."),
    Document(page_content="Кошки — независимые животные, которым нужно собственное пространство."),
    Document(page_content="Попугаи — умные птицы, способные имитировать человеческую речь."),
]

# Инициализация модели эмбеддингов
embeddings = GigaChatEmbeddings(
    verify_ssl_certs=False,
    scope="GIGACHAT_API_CORP",
)

# Создание векторного хранилища
vectorstore = Chroma.from_documents(documents, embeddings)


retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # выбор наиболе подходящего результата

retriever.batch(["кошка", "акула"])

In [ ]:
message = """
Отвечай на вопросы только с помощью полученного контекста.

{question}

Контекст:
{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [("human", message), ("assistant", "не могу тебе помочь"), ("human", "почему?")]
)

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model


##### Вызываем


In [ ]:
query = "Опиши кошек двумя словами. Используй функцию для получения дополнительной информации."
response_message = await rag_chain.ainvoke(query)

print("Ответ ассистента:", response_message.content)

[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/embeddings "HTTP/1.1 200 OK"
[INFO](httpx): HTTP Request: POST https://ngw.devices.sberbank.ru:9443/api/v2/oauth "HTTP/1.1 200 OK"
[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


Ответ ассистента: Не могу предоставить дополнительную информацию или проанализировать документ напрямую без использования специальных навыков, поскольку нужные инструменты сейчас недоступны. 

Однако исходя из предоставленного контекста: **независимые** и **своеобразные**.


Диалог нужно так же отправить вручную через `.flush()`


In [ ]:
tracer.dialog

[ChatMessage(role='user', content="\nОтвечай на вопросы только с помощью полученного контекста.\n\nОпиши кошек двумя словами. Используй функцию для получения дополнительной информации.\n\nКонтекст:\n[Document(id='26497d3d-920d-4050-a347-9c6eaad5a42c', metadata={}, page_content='Кошки — независимые животные, которым нужно собственное пространство.')]\n", function_call=None, function_result=None),
 ChatMessage(role='assistant', content='не могу тебе помочь', function_call=None, function_result=None),
 ChatMessage(role='user', content='почему?', function_call=None, function_result=None),
 ChatMessage(role='assistant', content='Не могу предоставить дополнительную информацию или проанализировать документ напрямую без использования специальных навыков, поскольку нужные инструменты сейчас недоступны. \n\nОднако исходя из предоставленного контекста: **независимые** и **своеобразные**.', function_call=None, function_result=None)]

In [ ]:
await tracer.flush()

[INFO](tagme_gigachain.tracer): Sending 4 chat messages (thread_id=856f351b-66c7-4027-b8d3-d7dfec708299)


### 3. Использование декораторов на примере цепочки RAG для генерации ответа модели с использованием RAG <a name="wrappers"></a>

Тут уже не используем tracer, callbacks у GigaChat оставляем пустым


In [ ]:
model = GigaChat(
    credentials=environ.get("GIGA_TOKEN"), verify_ssl_certs=False, scope="GIGACHAT_API_CORP", callbacks=[]
)

#### 3.1 Асинхронный декоратор


In [ ]:
from tagme_ai_traces import tagme_trace_async


@tagme_trace_async(
    token=token,  # можем передать токен или использовать env TAGME_TOKEN
    ssl=False,
    # tagme_client=tagme_async_client,  # можем вместо токена передать уже созданный клиент для отправки запросов к TagMe. Тогда декоратор не создаст новый клиент
    metadata={
        "experiment": "#1"
    },  # можем передать дополнительные метаданные. Базово берутся метаданные из `response_metadata` объектов диалога
    base_url=base_url,  # может быть передан base_url или использовать env TAGME_BASE_URL,
    # так же можем передать *args и **kwargs - они будут переданы в http-клиент. Список аргументов можно посмотреть в самих клиентах
)
async def run_model_async(input: LanguageModelInput) -> BaseMessage:
    return await model.ainvoke(input)


rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | run_model_async

response = cast(BaseMessage, await rag_chain.ainvoke("Опиши кошек двумя словами"))

response.content

[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/embeddings "HTTP/1.1 200 OK"
[INFO](httpx): HTTP Request: POST https://ngw.devices.sberbank.ru:9443/api/v2/oauth "HTTP/1.1 200 OK"
[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


'Потому что, согласно заданному условию, я должен отвечать исключительно на основе представленного контекста, а там указано лишь одно слово: «независимые». Других сведений, позволяющих добавить ещё одно слово, в данном контексте нет.'

Если в выводе нет никаких сообщений об ошибке, то всё прошло успешно


#### 3.2. Синхронный декоратор


In [ ]:
from tagme_ai_traces import tagme_trace


@tagme_trace(
    metadata={"experiment": "#2"},
)
def run_model_sync(input: LanguageModelInput) -> BaseMessage:
    return model.invoke(input)


rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | run_model_sync

response = rag_chain.invoke("Опиши кошек одним словом")

response.content

[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/embeddings "HTTP/1.1 200 OK"
[INFO](httpx): HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


'Потому что в предоставленном контексте содержится всего одна фраза: «Кошки — независимые животные, которым нужно собственное пространство». Из этой информации невозможно однозначно описать кошек одним словом. Нужна дополнительная информация или расширенный контекст.'

#### Функция для преобразования диалога

Дополнительно, если хотим использовать декоратор в неожиданном месте, можно передавать в него аргумент `dialog_transform_fc` - функцию, принимающую три позиционных аргумента: `model_input, model_response, metadata` и возвращающую `DialogData`

`model_input, model_response` - то, что приходит в wrapped и уходит из неё, соответственно

`metadata` - что мы сами кладём в декоратор, как в примере с `metadata={"experiment": "#1"}`